# Introduction to network analytics with Python
### With applications in actuarial science

## Basic Network Features

### Random Network

We start with generating a random Barabási-Albert network to show how to construct such random networks. Afterwards, we analyse its properties and calculate some network features. This allows us to illustrate the meaning of the *scale-free* property. 

In this part of the example, we will rely on the `igraph` library to generate the random network and calculate the metrics. The network itself will consist of 2000 nodes. The generator adds them one at a time. Additionally, we specify that each new node should have an edge to one other nodes. This end point for the new edge will be chosen randomly from the nodes already present, taking into account the degree of the other nodes (preferential attachment). 

In [32]:
import igraph as ig # to handle graph data and provide calculations and visualizations
import numpy as np # to handle numerical data
import pandas as pd # to handle tabular data
import matplotlib.pyplot as plt # to provide visualizations

In [ ]:
# Load the graph
graph = ig.Graph()

# Create the Barabasi-Albert network
# The graph should contain 200 nodes, with each new node connecting to one other
ba_graph = graph.Barabasi(200, m=1) 

print("Number of nodes: ", ba_graph.vcount())
print("Number of edges: ", ba_graph.ecount())

The `igraph` package allows us to quickly and easily calculate a whole range of metrics for this network. In this example, we will stick to four of them, in line with the ones discussed in the slide:
* **Degree**: number of connections for each node; 
* **Betweenness centrality**: relative number of shortest paths passing through this node;
* **Closeness centrality**: the inverse of the average distance to the other nodes;
* **PageRank**: importance of the node based on number of connections and importance neighbours.

In [34]:
degree = ba_graph.degree() #Degree
betw = ba_graph.betweenness() #Betweenness
cls = ba_graph.closeness() #Closeness
pgrnk = ba_graph.pagerank() #PageRank

We start with plotting the distributions of the four metrics. 

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15,10))
ax[0,0].hist(degree, bins=10)
ax[0,0].set_title('Degree Distribution')
ax[0,1].hist(betw, bins=10)
ax[0,1].set_title('Betweenness')
ax[1,0].hist(cls, bins=10)
ax[1,0].set_title('Closeness')
ax[1,1].hist(pgrnk, bins=10)
ax[1,1].set_title('PageRank')
plt.show()

First, we plot the degree distribution. The distribution of a Barabasi-Albert network exhibits the scale-free property. A visual test for this consists of plotting the density against the degree with both axes on a log-scale. The scale-free property is likely present when the plot resembles a straight line. 

This result is often present in real-world networks. It means that many nodes have only very few connection, while there are a limit number of nodes with a very high number of connections (called hubs).

In [36]:
def Pk(degree, values):
    num_nodes = len(values)
    count_degree = values.count(degree)
    density = count_degree/num_nodes
    return density

In [ ]:
list_degrees = range(min(degree), max(degree)+1)
list_densities = []
for i in list_degrees:
    density = Pk(i, degree)
    list_densities.append(density)

plt.scatter(list_degrees, list_densities)
plt.xlabel('Degree (log scale)')
plt.ylabel('Density (log scale)')
plt.title('Degree Distribution')
plt.xscale('log')
plt.yscale('log')

A general characteristic of networks, is that there is a strong correlation between the degree and the betweenness centrality. This will be tested and shown in the next piece of code. 

In [ ]:
plt.scatter(degree, betw)
plt.xlabel('Degree')
plt.ylabel('Betweenness')
plt.title('Betweenness vs Degree')

In [ ]:
plt.scatter(degree, betw)
plt.xlabel('Degree (log scale)')
plt.ylabel('Betweenness (log scale)')
plt.title('Betweenness vs Degree')
plt.xscale('log')
plt.yscale('log')

In the next part, we show how to visualise the network. Additionally, we will change the size of the node depending on the betweenness centrality. 

In [ ]:
# Calculate the layout
layout_fr = ba_graph.layout("fr")

#Define style from network plotting
visual_style = {}
visual_style["vertex_size"] = 5
visual_style["vertex_color"] = "red"
visual_style["layout"] = layout_fr
visual_style["bbox"] = (600, 600)
visual_style["margin"] = 20

ig.plot(ba_graph, **visual_style)

In [41]:
num_nodes = ba_graph.vcount()
ba_graph.vs['vertex_size'] = [5]
for i in range(num_nodes):
    betw_node = betw[i] + 1 # Avoid 0
    ba_graph.vs[i]['vertex_size'] = np.log(betw_node)*1.5 +1 # Avoid that 1 maps to 0 after log

In [ ]:
# Calculate the layout
layout_fr = ba_graph.layout("fr")

#Define style from network plotting
visual_style = {}
visual_style["vertex_size"] = ba_graph.vs['vertex_size']
visual_style["vertex_color"] = "red"
visual_style["layout"] = layout_fr
visual_style["bbox"] = (600, 600)
visual_style["margin"] = 20

ig.plot(ba_graph, **visual_style)

As we have seen in the slides, in most networks, only a fraction of the possible edges are actually present in the network. This results in a very sparse adjacency matrix (containing mostly 0). To illustrate this, we are going to visualise the adjacency matrix of the random network below. A black pixel signifies that there is an edge, while white space means that there is no connection between the two nodes. 

In [ ]:
adjacency_matrix = ba_graph.get_adjacency()
plt.imshow(adjacency_matrix, cmap='binary', interpolation='none')

### Health Care Provider Network

We are going to move to the data set that will also be used later in this notebook when constructing a classification model. The data itself is taken from Kaggle, and consists of health care insurance claims, with information of the claims and patients involved. 
The claim data is split into two data sets (which we will combine for ease of use): in-patient data and out-patient data. These contain patient data for patient admitted in the hospital and those who aren't, respectively. 

The set-up of the data is to uncover those health care providers that are indulging in Medicare fraud.

In the following pieces of code, we are going to do a first analysis of the data. We will construct a network containing the following four types of nodes (heterogeneous network):
* Providers
* Physicians
* Patients
* Claims

In [44]:
claims_inpatients = pd.read_csv('data/Train_Inpatientdata-1542865627584.csv')
claims_outpatients = pd.read_csv('data/Train_Outpatientdata-1542865627584.csv')
patient_data = pd.read_csv('data/Train_Beneficiarydata-1542865627584.csv')
labels = pd.read_csv('data/Train-1542865627584.csv')

In [ ]:
claims_inpatients.head()

In [ ]:
claims_outpatients.head()

In [ ]:
patient_data.head()

In [ ]:
labels.head()

As a first pre-processing step, we are going to convert the fraud labels into binary labels, i.e., 0 and 1. Here, 1 will indicate that the provider is potentially fraudulent. 

In [ ]:
map_label = {'No': 0, 'Yes': 1}
labels['PotentialFraud'] = labels['PotentialFraud'].map(map_label)
labels.head()

It is now easy to do a first analysis of the prevalence of the fraud labels. 

In [ ]:
prevalence = labels['PotentialFraud'].mean()
print("Prevalence of fraud: ", round(prevalence*100, 2), "%")

The network itself will be purely based on the claim information. So we set up a claim as central node, and link it to the provider, patient an attending physician. Note that it is possible to also add the other physicians involved, but we do not do it here for simplicity. 

For this example, we will work with a subset of claims. We will take 100 claims for the inpatients and 100 for the outpatients, to have some clear visualisations. The concepts and code stay the same larger networks.

In [ ]:
from tqdm import tqdm
# Create a new graph
healthcare_graph = ig.Graph()

# Take first 1000 rows for inpatients and outpatients
claims_inpatients = claims_inpatients.iloc[:100]
claims_outpatients = claims_outpatients.iloc[:100]

# Add nodes for providers, physicians, patients, and claims
providers = list(np.unique(claims_inpatients['Provider'].tolist() + claims_outpatients['Provider'].tolist()))
physicians = list(np.unique(claims_inpatients['AttendingPhysician'].tolist() + claims_outpatients['AttendingPhysician'].tolist()))
patients = list(np.unique(claims_inpatients['BeneID'].tolist() + claims_outpatients['BeneID'].tolist()))
claims = list(np.unique(claims_inpatients['ClaimID'].tolist() + claims_outpatients['ClaimID'].tolist()))

# Add vertices to the graph
healthcare_graph.add_vertices(providers + physicians + patients + claims)

# Add edges based on the relationships in the data
edges = []

# Add edges for inpatients
for _, row in claims_inpatients.iterrows():
    edges.append((row['Provider'], row['ClaimID']))
    edges.append((row['AttendingPhysician'], row['ClaimID']))
    edges.append((row['BeneID'], row['ClaimID']))

# Add edges for outpatients
for _, row in claims_outpatients.iterrows():
    edges.append((row['Provider'], row['ClaimID']))
    edges.append((row['AttendingPhysician'], row['ClaimID']))
    edges.append((row['BeneID'], row['ClaimID']))

# Add edges to the graph
edges_filtered = []
for edge in tqdm(edges):
    # Test if none of the two elements is NaN
    if edge[0] == edge[0] and edge[1] == edge[1]:
        edges_filtered.append(edge)

healthcare_graph.add_edges(edges_filtered)

# Print summary of the graph
print(healthcare_graph.summary())

In [ ]:
# Define colours depending on type of node
num_nodes = healthcare_graph.vcount()
healthcare_graph.vs['vertex_color'] = ["red"]*num_nodes

for i in tqdm(range(num_nodes)):
    name = healthcare_graph.vs[i]['name']
    if name in providers:
        healthcare_graph.vs[i]['vertex_color'] = "blue"
    elif name in physicians:
        healthcare_graph.vs[i]['vertex_color'] = "green"
    elif name in patients:
        healthcare_graph.vs[i]['vertex_color'] = "yellow"
    elif name in claims:
        healthcare_graph.vs[i]['vertex_color'] = "purple"

# Calculate the layout
layout_fr = healthcare_graph.layout("fr")

#Define style from network plotting
visual_style = {}
visual_style["vertex_size"] = 4
visual_style["vertex_color"] = healthcare_graph.vs['vertex_color']
visual_style["layout"] = layout_fr
visual_style["bbox"] = (600, 600)
visual_style["margin"] = 20

In [ ]:
ig.plot(healthcare_graph, **visual_style)

## Representation Learning